# Khám phá dữ liệu, tiền xử lý, phân tích đơn giản

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.neural_network import MLPClassifier
from sklearn import set_config
set_config(display='diagram') # Để trực quan hóa pipeline

### Đọc dữ liệu từ file

In [2]:
air_df = pd.read_csv('data.csv', sep = ',')

In [3]:
air_df.head()

,co,no,no2,o3,so2,pm2_5,pm10,nh3,aqi
0,323.77,0.21,5.74,17.35,3.46,4.60,6.11,2.53,1
1,343.80,0.78,5.66,18.60,3.76,4.66,6.36,2.79,1
2,353.81,0.91,6.00,20.92,4.05,4.54,6.36,3.14,1
3,360.49,0.71,6.08,24.68,4.23,4.37,6.17,3.29,1
4,340.46,0.48,5.14,30.04,4.35,4.41,5.81,2.98,1


### Dữ liệu có bao nhiêu dòng và bao nhiêu cột?

In [4]:
air_df.shape

(4422, 9)

### Mỗi dòng có ý nghĩa gì? Có vấn đề các dòng có ý nghĩa khác nhau không?

Mỗi dòng trong DataFrame `air_df` cho biết dữ liệu về các chỉ số và chất lượng không khí của các tỉnh, thành phố từ 0h ngày 1/8/2021 - 0h ngày 1/9/2021 . Có vẻ không có vấn đề các dòng có ý nghĩa khác nhau.

### Dữ liệu có các dòng bị lặp không?

In [5]:
air_df.index.duplicated().sum()

0

### Mỗi cột có ý nghĩa gì?

In [6]:
with open('description.txt', 'r',encoding='utf-8') as f:
    print(f.read())

VARIABLE DESCRIPTIONS:
aqi        	Chỉ số chất lượng không khí (1,2,3,4,5)
                (1 = Good; 2 = Fair; 3 = Moderate; 4 = Poor ; 5 = Very Poor )
co          	Nồng độ CO (Carbon monoxide) 		,  μg/m3
no            	Nồng độ NO (Nitrogen monoxide) 		,  μg/m3
no2             Nồng độ NO2 (Nitrogen dioxide) 		,  μg/m3
o3             	Nồng độ O3 (Ozone) 			,  μg/m3
so2           	Nồng độ SO2 (Sulphur dioxide) 		,  μg/m3
pm2_5           Nồng độ PM2.5 (Fine particles matter) 	,  μg/m3
pm10          	Nồng độ PM10 (Coarse particulate matter),  μg/m3
nh3            	Nồng độ NH3 (Ammonia) 			,  μg/m3




## Đưa ra câu hỏi cần trả lời

- Chất lượng không khí được tính như thế nào từ các chỉ số trong không khí đã được đo từ môi trường ?
<br/>
- Ý nghĩa thực tế của câu hỏi: Theo Tổ chức Y tế thế giới (WHO), hằng năm có 34.000 ca tử vong tại Việt Nam có liên quan đến ô nhiễm không khí. Việc xác định chất lượng không khí giúp người dân có các biện pháp bảo vệ bản thân và cộng đồng kịp thời như đeo khẩu trang, hạn chế sử dụng các phương tiện giao thông gây ô nhiễm, ... . Ngoài ra, chính quyền địa phương có thể hạn chế lượng khí thải từ các nhà máy, xí nghiệp, trồng thêm nhiều cây xanh để làm giảm ô nhiễm không khí.

# Khám phá dữ liệu

In [7]:
# Cột output hiện có kiểu dữ liệu gì?
air_df['aqi'].dtype

dtype('int64')

In [8]:
# Cột output có bao nhiêu giá trị thiếu?
air_df['aqi'].isna().sum()

0

In [9]:
# Tỉ lệ các lớp trong cột output?
air_df['aqi'].value_counts(normalize=True) * 100

1    37.697874
2    24.310267
5    17.254636
4    14.540932
3     6.196291
Name: aqi, dtype: float64

- Cột output đang ở kiểu dạng số, không có giá trị thiếu và tỉ lệ lớp 3 trong cột output chênh lệch so với các lớp còn lại, tuy nhiên không nhiều

## Tiền xử lý (tách các tập)

In [10]:
# Tách X và y
y_sr = air_df["aqi"] # sr là viết tắt của series
X_df = air_df.drop("aqi", axis=1)

In [11]:
# Tách tập huấn luyện,tập validation, tập test theo tỉ lệ 70%:20%:10%
temp_X_df, test_X_df, temp_y_sr, test_y_sr = \
                              train_test_split(X_df, y_sr, 
                                               test_size=0.1, 
                                               stratify=y_sr, 
                                               random_state=60)
train_X_df, val_X_df, train_y_sr, val_y_sr = \
                              train_test_split(temp_X_df, temp_y_sr, 
                                               test_size=2/9, 
                                               stratify=temp_y_sr, 
                                               random_state=60)


In [12]:
train_X_df.shape

(3094, 8)

In [13]:
train_y_sr.shape

(3094,)

In [14]:
val_X_df.shape

(885, 8)

In [15]:
val_y_sr.shape

(885,)

In [16]:
test_X_df.shape

(443, 8)

In [17]:
test_y_sr.shape

(443,)

In [18]:
train_X_df.head().index

Int64Index([759, 2017, 570, 344, 1358], dtype='int64')

# Khám phá dữ liệu (tập huấn luyện)

### Mỗi cột input hiện đang có kiểu dữ liệu gì? Có cột nào có kiểu dữ liệu chưa phù hợp để có thể xử lý tiếp không?

In [19]:
train_X_df.dtypes

co       float64
no       float64
no2      float64
o3       float64
so2      float64
pm2_5    float64
pm10     float64
nh3      float64
dtype: object

- Các cột đều có dữ liệu phù hợp

### Với mỗi cột input có kiểu dữ liệu dạng số, các giá trị được phân bố như thế nào?

In [20]:
def missing_percentage(c):
    return (c.isna().mean() * 100).round(1)
def median(c):
    return c.quantile(0.5).round(1)
train_X_df.agg([missing_percentage, 'min', median, 'max'])

,co,no,no2,o3,so2,pm2_5,pm10,nh3
missing_percentage,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
min,203.61,0.00,0.17,0.00,0.04,0.50,0.54,0.00
median,460.60,0.10,6.20,21.50,2.70,14.40,16.60,1.60
max,14312.74,343.32,157.65,211.72,225.07,811.92,844.52,102.34


## Tiền xử lý (tập huấn luyện)

- Với các cột dạng số, ta sẽ điền giá trị thiếu bằng giá trị mean của cột <font color=gray>(gợi ý: dùng `SimpleImputer` trong Sklearn)</font>. Với *tất cả* các cột dạng số trong tập huấn luyện, ta đều cần tính mean, vì ta không biết được cột nào sẽ bị thiếu giá trị khi dự đoán với các véc-tơ input mới. 

- Cuối cùng, khi tất cả các cột đã được điền giá trị thiếu và đã có dạng số, ta sẽ tiến hành chuẩn hóa bằng cách trừ đi mean và chia cho độ lệch chuẩn của cột để giúp cho các thuật toán cực tiểu hóa như Gradient Descent, LBFGS, ... hội tụ nhanh hơn <font color=gray>(gợi ý: dùng `StandardScaler` trong Sklearn)</font>.

In [21]:
cols=list(train_X_df.columns)
temp=make_column_transformer(
        (SimpleImputer(strategy='mean'),cols))
preprocess_pipeline=make_pipeline(temp,StandardScaler())
preprocessed_train_X=preprocess_pipeline.fit_transform(train_X_df)

In [22]:
preprocess_pipeline

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('simpleimputer',
                                                  SimpleImputer(),
                                                  ['co', 'no', 'no2', 'o3',
                                                   'so2', 'pm2_5', 'pm10',
                                                   'nh3'])])),
                ('standardscaler', StandardScaler())])

## Tiền xử lý (tập validation)

In [23]:
preprocessed_val_X=preprocess_pipeline.transform(val_X_df)

## Tiền xử lý + mô hình hóa